In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
import pickle
from time import time
import tqdm

import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

from op_fourier_trafo_unitary import operator_fourier_circuit, brute_prepare_gaussian_state, \
    inverse_operator_fourier_circuit
from boltzmann import lookup_table_boltzmann, inverse_lookup_table_boltzmann
from tools.classical import *
from tools.quantum import *


In [12]:
distances = []
for _ in tqdm.tqdm(range(10)):
    # np.random.seed(667)
    num_qubits = 3
    num_energy_bits = 4
    bohr_bound = 0 # 2 ** (-num_energy_bits + 1) #!
    eps = 0.1
    sigma = 5
    eig_index = np.random.randint(2 ** num_qubits)
    print(f"eig_index: {eig_index}")
    T = 1
    shots = 1
    delta = 0.001

    hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
    rescaled_coeff = hamiltonian.rescaled_coeffs
    # Corresponding Trotter step circuit
    trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, disordering=True)
    inverse_trotter_step_circ = inverse_trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, disordering=True)
    hamiltonian.trotter_step_circ = trotter_step_circ
    hamiltonian.inverse_trotter_step_circ = inverse_trotter_step_circ

    initial_state = hamiltonian.eigenstates[:, eig_index]
    initial_state = Statevector(initial_state)

    qr_delta = QuantumRegister(1, name='delta')
    qr_boltzmann = QuantumRegister(1, name='boltz')
    qr_energy = QuantumRegister(num_energy_bits, name="w")
    qr_sys  = QuantumRegister(num_qubits, name="sys")

    circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys)
    U_circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys, name='U')
    U_dag_circ = QuantumCircuit(qr_boltzmann, qr_energy, qr_sys, name='U_dag')
    
    if sigma != 0.:
        prep_circ = brute_prepare_gaussian_state(num_energy_bits, sigma)
        gauss_state = Statevector(prep_circ)
        circ.compose(prep_circ, qr_energy, inplace=True)
    else:  # Conventional QPE
        circ.h(qr_energy)
    
    # System prep
    circ.initialize(initial_state, qr_sys)

    jump_op = Operator(Pauli('X'))
    #* --- U
    # OFT
    oft_circ = operator_fourier_circuit(jump_op, num_qubits, num_energy_bits, hamiltonian, 
                                        initial_state=initial_state)
    U_circ.compose(oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)

    # W
    boltzmann_circ = lookup_table_boltzmann(num_energy_bits)
    U_circ.compose(boltzmann_circ, [qr_boltzmann[0], *list(qr_energy)], inplace=True)

    #* --- U_dag
    # W
    boltzmann_circ_inverse = inverse_lookup_table_boltzmann(num_energy_bits)
    U_dag_circ.compose(boltzmann_circ_inverse, [qr_boltzmann[0], *list(qr_energy)], inplace=True)

    # OFT
    inverse_oft_circ = inverse_operator_fourier_circuit(jump_op, num_qubits, num_energy_bits, hamiltonian)
    U_dag_circ.compose(inverse_oft_circ, [*list(qr_energy), *list(qr_sys)], inplace=True)

    circ.compose(U_circ, [qr_boltzmann[0], *list(qr_energy), *list(qr_sys)], inplace=True)
    circ.compose(U_dag_circ, [qr_boltzmann[0], *list(qr_energy), *list(qr_sys)], inplace=True)
    
    end_state_UUdag = Statevector(circ)
    
    boltz_initial_state = Statevector(np.array([1, 0]))
    full_initial_state = boltz_initial_state.expand(gauss_state).expand(initial_state).data
    dist_to_initial_state = np.linalg.norm(full_initial_state - end_state_UUdag.data)
    print(f'Basic distance {dist_to_initial_state}')
    
    fid = state_fidelity(full_initial_state, end_state_UUdag)
    print(f'Fidelity: {fid}')
    
    
    # U_op = Operator(U_circ).data
    # U_dag_op = Operator(U_dag_circ).data
    # dist_to_id = np.linalg.norm(U_op @ U_dag_op - np.eye(2 ** (num_qubits + num_energy_bits + 1)))
    # distances.append(dist_to_id)


  0%|          | 0/10 [00:00<?, ?it/s]

eig_index: 6
Original spectrum:  [-4.     -3.4641 -2.8284 -2.      2.      2.8284  3.4641  4.    ]
Ideal spectrum:  [-0.      0.0301  0.0659  0.1125  0.3375  0.3841  0.4199  0.45  ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaled coefficients:  [0.05625 0.05625 0.05625 0.05625]
Energy before jump: 0.41985571585149867
Jump applied to 2th qubit
Energy after jump: 0.35490381056766684
Energy jump: -0.06495190528383182
Inverse jump applied to 2th qubit


 10%|█         | 1/10 [00:07<01:10,  7.85s/it]

Basic distance 1.2279142402809964
Fidelity: 0.06057176099374703
eig_index: 7
Original spectrum:  [-4.     -3.4641 -2.8284 -2.      2.      2.8284  3.4641  4.    ]
Ideal spectrum:  [-0.      0.0301  0.0659  0.1125  0.3375  0.3841  0.4199  0.45  ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaled coefficients:  [0.05625 0.05625 0.05625 0.05625]
Energy before jump: 0.44999999999999996
Jump applied to 2th qubit
Energy after jump: 0.22499999999999998
Energy jump: -0.22499999999999998
Inverse jump applied to 2th qubit


 10%|█         | 1/10 [00:14<02:12, 14.68s/it]


KeyboardInterrupt: 